In [14]:
#!pip install openai -q
#!pip install pandas -q
#!pip install datasets -q
#!pip install langchain -q
#!pip3 install scikit-learn scipy matplotlib -q

In [1]:
import pandas as pd
import os
import requests
import tarfile
import time

from datasets import load_dataset
from langchain import PromptTemplate, OpenAI, LLMChain
from getpass import getpass
from sklearn.metrics import classification_report, confusion_matrix

## Dataset: IMDB Movie Review
For this project, we'll use the [IMDB Movie Review dataset](https://huggingface.co/datasets/imdb), available on HuggingFace.

In [2]:
dataset = load_dataset("imdb", split = 'train')
dataset
# Print the first sample in the dataset
print(dataset[0])


Found cached dataset imdb (C:/Users/crist/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [3]:
# Print the text and label of the first sample
print("Text:", dataset[0]["text"])
print("Label:", dataset[0]["label"])
## creating a code to print sample data from the dataset    

Text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

In [5]:
import openai
import random


# Read the API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set")

# Replace this with your own OpenAI API key
openai.api_key = api_key

# Load the IMDB Movie Review dataset using Hugging Face's datasets library
dataset = load_dataset("imdb", split="train")

# Function to perform sentiment analysis using ChatGPT
def analyze_sentiment(review):
    prompt = f"Analyze the sentiment of the following movie review: '{review}'"

    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=30,
        n=1,
        stop=None,
        temperature=0.5,
    )

    sentiment = response.choices[0].text.strip()
    return sentiment.lower()

# Randomly sample 10 instances from the dataset
sample_size = 10
sample_indices = random.sample(range(len(dataset)), sample_size)
sampled_dataset = dataset.select(sample_indices)

Found cached dataset imdb (C:/Users/crist/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [22]:
def analyze_sentiment(review):
    prompt = f"Analyze the sentiment of the following movie review: '{review}'"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that analyzes the sentiment of movie reviews."},
            {"role": "user", "content": f"Analyze this review: '{review}'"}
        ],
        max_tokens=30,
        n=1,
        temperature=0.5,
    )

    sentiment = response['choices'][0]['message']['content'].strip()
    return sentiment.lower()

In [23]:
# Perform sentiment analysis on the sampled instances
sentiment_results = []
for example in sampled_dataset:
    sentiment = analyze_sentiment(example["text"])
    sentiment_results.append(sentiment)

# Add the sentiment analysis results to the sampled dataset
sampled_dataset = sampled_dataset.add_column("sentiment", sentiment_results)

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

In [24]:
# Print the dataset with headers
print("Index\t", "Review\t", "Label\t", "Sentiment")

# Iterate through the dataset and print each row
for i, example in enumerate(sampled_dataset):
    print(f"{i}\t", example["text"][:50], "...", "\t", example["label"], "\t", example["sentiment"])

Index	 Review	 Label	 Sentiment
0	 It is written in stone that Disney animations simp ... 	 1 	 overall, the sentiment of this review is positive. the reviewer enjoyed the fact that the movie was not a typical disney musical and found the story to
1	 This is an evocative and idealized portrait of the ... 	 1 	 based on the language used in this review, it seems to be a positive one. the reviewer describes the movie as "excellent" and "
2	 I love this anime! I was laughing my head off with ... 	 1 	 overall, the sentiment of this review is positive. the reviewer expresses their love for the anime and enjoys the humor and slapstick violence. the characters
3	 The "movie aimed at adults" is a rare thing these  ... 	 1 	 overall, this review has a positive sentiment towards the movie moonstruck. the reviewer praises the fact that the movie is aimed at adults and does
4	 Yes, he is! ...No, not because of Pintilie likes t ... 	 0 	 the sentiment of this review is negative. the reviewer expre